# Imports

In [1]:
import os
import wandb
import pandas as pd

from pathlib import Path

import project_config as pc

# Retrieve dataset and model

In [ ]:
# Init run
run = wandb.init(project=pc.WANDB_PROJECT,
				 entity=pc.WANDB_ENTITY,
				 dir=pc.WANDB_LOCAL_LOGS_PATH,
				 job_type='inference')

# Download latest dataset version (if not already downloaded)
dataset_artifact = run.use_artifact(f'{pc.DATASET_ARTIFACT_NAME}:latest')
dataset_dir = pc.WANDB_LOCAL_ARTIFACTS_PATH+Path(dataset_artifact._default_root()).stem
if not os.path.exists(dataset_dir):
	_ = dataset_artifact.download(root=dataset_dir)

# Download latest model version (if not already downloaded)
model_artifact = run.use_artifact('model:latest')
model_dir = pc.WANDB_LOCAL_ARTIFACTS_PATH+pc.DATASET_ARTIFACT_NAME+'-'+Path(model_artifact._default_root()).stem
if not os.path.exists(model_dir):
	_ = model_artifact.download(root=model_dir)

# Finish run
run.finish()

# Inference

In [ ]:
from DLOlympus.fastai.inference import FastAIInferencer

# Create inferencer
inferencer = FastAIInferencer(model_dir+'/model.pkl')
inferencer.init()

# Get items
df = pd.read_csv(dataset_dir+'/dataset.csv')
df = df[df['is_valid']]
items = df['file_path'].apply(lambda x: dataset_dir+'/'+x).values

# Get predictions
raw_preds, decoded_preds, outputs = inferencer.process(items)

In [ ]:
outputs

In [ ]:
from sklearn.metrics import f1_score

# Check metric
f1_score(outputs[0], df['label_breed'].values, average='macro')

# ONNX

In [17]:
import albumentations

c, h, w = 3, 64, 64 

stats_mean = [0.485, 0.456, 0.406]
stats_std = [0.229, 0.224, 0.225]

transforms = albumentations.Compose([
	albumentations.SmallestMaxSize(max_size=max(h,w)),
	albumentations.CenterCrop(height=h, width=w),
	albumentations.Normalize(stats_mean, stats_std),
	albumentations.ToTensorV2()
])

In [18]:
# fastai 2 onnx conversion
inferencer.export([c, h, w])

In [ ]:
from DLOlympus.onnx.inference import ONNXInferencer

# Create inferencer
inferencer = ONNXInferencer('model.onnx')
inferencer.init(transforms)

onnx_outputs = inferencer.process(items, batch_size=256)